# TIME TRAVEL

In [0]:
%sql

CREATE OR REPLACE TABLE time_travel_table (
  id INT,
  name STRING,
  gender STRING,
  salary integer,
  dept STRING
) USING DELTA
LOCATION 'dbfs:/FileStore/tables/delta/time_travel';

In [0]:
%sql

INSERT INTO time_travel_table VALUES (1, "Carl Mike", "m", 170000, "Support");

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql

INSERT INTO time_travel_table (id, name, gender, salary, dept) VALUES
(4, "Mary Scala", "f", 230000, "Sales");

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql

INSERT INTO time_travel_table (id, name, gender, salary, dept) VALUES
(2, "Mikel Clark", "m", 254300, "IT"),
(3, "Bob Smith", "m", 220000, "IT");

num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql

INSERT INTO time_travel_table (id, name, gender, salary, dept) VALUES
(5, "Susan Liam", "f", 150000, "Sales");

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql

DELETE FROM time_travel_table WHERE id=5;

num_affected_rows
1


In [0]:
%sql

SELECT * FROM time_travel_table;

id,name,gender,salary,dept
2,Mikel Clark,m,254300,IT
3,Bob Smith,m,220000,IT
1,Carl Mike,m,170000,Support
4,Mary Scala,f,230000,Sales


## Table History

In [0]:
%sql

DESCRIBE HISTORY time_travel_table;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
5,2024-08-26T16:13:43.000+0000,8904480319608266,andresmunozpampillonaws@gmail.com,DELETE,"Map(predicate -> [""(id#3169 = 5)""])",null,List(3993455990136351),0826-160231-sfl7d1de,4,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 1539, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 4284, numDeletedRows -> 1, scanTimeMs -> 3136, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 1130)",null,Databricks-Runtime/12.2.x-scala2.12
4,2024-08-26T16:13:12.000+0000,8904480319608266,andresmunozpampillonaws@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(3993455990136351),0826-160231-sfl7d1de,3,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1539)",null,Databricks-Runtime/12.2.x-scala2.12
3,2024-08-26T16:10:08.000+0000,8904480319608266,andresmunozpampillonaws@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(3993455990136351),0826-160231-sfl7d1de,2,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 2, numOutputBytes -> 1582)",null,Databricks-Runtime/12.2.x-scala2.12
2,2024-08-26T16:09:59.000+0000,8904480319608266,andresmunozpampillonaws@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(3993455990136351),0826-160231-sfl7d1de,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1539)",null,Databricks-Runtime/12.2.x-scala2.12
1,2024-08-26T16:08:40.000+0000,8904480319608266,andresmunozpampillonaws@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(3993455990136351),0826-160231-sfl7d1de,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1546)",null,Databricks-Runtime/12.2.x-scala2.12
0,2024-08-26T16:07:35.000+0000,8904480319608266,andresmunozpampillonaws@gmail.com,CREATE OR REPLACE TABLE,"Map(isManaged -> false, description -> null, partitionBy -> [], properties -> {})",null,List(3993455990136351),0826-160231-sfl7d1de,null,WriteSerializable,true,Map(),null,Databricks-Runtime/12.2.x-scala2.12


##Spark Timestamp Table Name Time Travel

In [0]:
df = spark.read.format("delta").option("timestampAsOf", "2024-08-26T16:10:08.000+00:00").table('time_travel_table')

display(df)

id,name,gender,salary,dept
2,Mikel Clark,m,254300,IT
3,Bob Smith,m,220000,IT
1,Carl Mike,m,170000,Support
4,Mary Scala,f,230000,Sales


##Spark Timestamp Table Path Time Travel

In [0]:
df = spark.read.format("delta").option("timestampAsOf", "2024-08-26T16:08:40.000+00:00").load('dbfs:/FileStore/tables/delta/time_travel')

display(df)

id,name,gender,salary,dept
1,Carl Mike,m,170000,Support


##Spark Version Table Name Time Travel

In [0]:
df = spark.read.format("delta").option("versionAsOf", 2).table('time_travel_table')

display(df)

id,name,gender,salary,dept
1,Carl Mike,m,170000,Support
4,Mary Scala,f,230000,Sales


##Spark Version Table Path Time Travel

In [0]:
df = spark.read.format("delta").option("versionAsOf", 4).load('dbfs:/FileStore/tables/delta/time_travel')

display(df)

id,name,gender,salary,dept
2,Mikel Clark,m,254300,IT
3,Bob Smith,m,220000,IT
1,Carl Mike,m,170000,Support
4,Mary Scala,f,230000,Sales
5,Susan Liam,f,150000,Sales


## SQL Timestamp Table Name Time Travel

In [0]:
%sql

SELECT * FROM time_travel_table TIMESTAMP AS OF "2024-08-26T16:10:08.000+00:00"

id,name,gender,salary,dept
2,Mikel Clark,m,254300,IT
3,Bob Smith,m,220000,IT
1,Carl Mike,m,170000,Support
4,Mary Scala,f,230000,Sales


## SQL Timestamp Table Path Time Travel

In [0]:
%sql

SELECT * FROM delta.`dbfs:/FileStore/tables/delta/time_travel` TIMESTAMP AS OF "2024-08-26T16:08:40.000+00:00"

id,name,gender,salary,dept
1,Carl Mike,m,170000,Support


## SQL Vesion Table Name Time Travel

In [0]:
%sql

SELECT * FROM time_travel_table VERSION AS OF 4

id,name,gender,salary,dept
2,Mikel Clark,m,254300,IT
3,Bob Smith,m,220000,IT
1,Carl Mike,m,170000,Support
4,Mary Scala,f,230000,Sales
5,Susan Liam,f,150000,Sales


## SQL Vesion Table Path Time Travel

In [0]:
%sql

SELECT * FROM delta.`dbfs:/FileStore/tables/delta/time_travel` VERSION AS OF 1

id,name,gender,salary,dept
1,Carl Mike,m,170000,Support
